In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [2]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

In [3]:
async def get_tools(params):
    async with MCPServerStdio(params=params) as server:
        tools = await server.list_tools()

    for tool in tools:
        print(f"{tool.name}: {tool.description.replace('\n',' ')}")

    return tools, server;

In [4]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params) as server:
    fetch_tools = await server.list_tools()

for tool in fetch_tools:
    print(f"{tool.name}: {tool.description.replace('\n',' ')}")

fetch: Fetches a URL from the internet and optionally extracts its contents as markdown.  Although originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.


### And now repeat for 3 more!

In [5]:
puppeteer_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-puppeteer"]}

puppeteer_tools, puppeteer_server =await get_tools(puppeteer_params)

puppeteer_navigate: Navigate to a URL
puppeteer_screenshot: Take a screenshot of the current page or a specific element
puppeteer_click: Click an element on the page
puppeteer_fill: Fill out an input field
puppeteer_select: Select an element on the page with Select tag
puppeteer_hover: Hover an element on the page
puppeteer_evaluate: Execute JavaScript in the browser console


In [6]:
puppeteer_tools[0].inputSchema

{'type': 'object',
 'properties': {'url': {'type': 'string', 'description': 'URL to navigate to'},
  'launchOptions': {'type': 'object',
   'description': "PuppeteerJS LaunchOptions. Default null. If changed and not null, browser restarts. Example: { headless: true, args: ['--no-sandbox'] }"},
  'allowDangerous': {'type': 'boolean',
   'description': 'Allow dangerous LaunchOptions that reduce security. When false, dangerous args like --no-sandbox will throw errors. Default false.'}},
 'required': ['url']}

In [7]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

file_tools, file_server = await get_tools(files_params)
# await file_server.connect()
# await file_server.call_tool("write_file", {"path": f"{sandbox_path}/test.txt", "content": "Hello, world!"})
# await file_server.call_tool("read_file", {"path": f"{sandbox_path}/test.txt"})


read_file: Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.
read_multiple_files: Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.
write_file: Create a new file or completely overwrite an existing file with new content. Use with caution as it will overwrite existing files without warning. Handles text content with proper encoding. Only works within allowed directories.
edit_file: Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showin

In [43]:
playwright_params = {"command":"npx",
                    "args":["-y", "@playwright/mcp@latest"],
                    "browser": "firefox",
                    "headless": False,
                    "slow_mo": 100,  # Optional: slow down actions for visibility
                    "devtools": True # Optional: open devtools
                    }

playwright_tools, playwright_server = await get_tools(playwright_params)
await server.connect()
await server.call_tool("browser_navigate", {"url": "https://www.google.com"})

browser_close: Close the page
browser_resize: Resize the browser window
browser_console_messages: Returns all console messages
browser_handle_dialog: Handle a dialog
browser_file_upload: Upload one or multiple files
browser_install: Install the browser specified in the config. Call this if you get an error about the browser not being installed.
browser_press_key: Press a key on the keyboard
browser_navigate: Navigate to a URL
browser_navigate_back: Go back to the previous page
browser_navigate_forward: Go forward to the next page
browser_network_requests: Returns all network requests since loading the page
browser_pdf_save: Save page as PDF
browser_take_screenshot: Take a screenshot of the current page. You can't perform actions based on the screenshot, use browser_snapshot for actions.
browser_snapshot: Capture accessibility snapshot of the current page, this is better than screenshot
browser_click: Perform click on a web page
browser_drag: Perform drag and drop between two elements
b

CallToolResult(meta=None, content=[TextContent(type='text', text='Contents of https://www.google.com/:\nGoogle offered in: [हिन्दी](https://www.google.com/setprefs?sig=0_VvFFhvofC8jTX3B9_ZkSyyaHp3Y%3D&hl=hi&source=homepage&sa=X&ved=0ahUKEwj4jIPtuumNAxW1a_UHHdUSLIkQ2ZgBCAY) [বাংলা](https://www.google.com/setprefs?sig=0_VvFFhvofC8jTX3B9_ZkSyyaHp3Y%3D&hl=bn&source=homepage&sa=X&ved=0ahUKEwj4jIPtuumNAxW1a_UHHdUSLIkQ2ZgBCAc) [తెలుగు](https://www.google.com/setprefs?sig=0_VvFFhvofC8jTX3B9_ZkSyyaHp3Y%3D&hl=te&source=homepage&sa=X&ved=0ahUKEwj4jIPtuumNAxW1a_UHHdUSLIkQ2ZgBCAg) [मराठी](https://www.google.com/setprefs?sig=0_VvFFhvofC8jTX3B9_ZkSyyaHp3Y%3D&hl=mr&source=homepage&sa=X&ved=0ahUKEwj4jIPtuumNAxW1a_UHHdUSLIkQ2ZgBCAk) [தமிழ்](https://www.google.com/setprefs?sig=0_VvFFhvofC8jTX3B9_ZkSyyaHp3Y%3D&hl=ta&source=homepage&sa=X&ved=0ahUKEwj4jIPtuumNAxW1a_UHHdUSLIkQ2ZgBCAo) [ગુજરાતી](https://www.google.com/setprefs?sig=0_VvFFhvofC8jTX3B9_ZkSyyaHp3Y%3D&hl=gu&source=homepage&sa=X&ved=0ahUKEwj4jIPtuu

### And now.. bring on the Agent with Tools!

In [44]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 74701a8c35f6: 100% ▕██████████████████▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [45]:
from agents import OpenAIChatCompletionsModel, AsyncOpenAI

# Create a client that talks to your local Ollama server
llm_client = AsyncOpenAI(
    base_url=os.getenv("OLLAMA_URL"),  # Ollama's OpenAI-compatible endpoint
    api_key=os.getenv("OLLAMA_API_KEY"),  # Any string works, it's not authenticated
)

# Specify the Ollama model you want to use
llm_model = OpenAIChatCompletionsModel(
    model="llama3.2:3b",  # or llama3, mistral, etc.
    openai_client=llm_client,
)

gemini_model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=AsyncOpenAI(
        base_url=os.getenv("GEMINI_URL"),  # Ollama's OpenAI-compatible endpoint
        api_key=os.getenv("GEMINI_API_KEY"),  # Any string works, it's not authenticated
    ),
)

In [ ]:
import asyncio
#System prompt
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task,
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another.
Be persistent until you have solved your assignment,
trying different options and sites as needed.
Save the output in markdown format to the file as asked by the user in the directory by using tools.
"""

async with MCPServerStdio(params=files_params, cache_tools_list=True) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, cache_tools_list=False) as mcp_server_browser:
        agent = Agent(
            name="investigator",
            instructions=instructions,
            model=gemini_model,
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(
                agent,
                "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee9.md using the tools provided"
                )
            print(result.final_output)

Error invoking MCP tool browser_install: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.


AgentsException: Error invoking MCP tool browser_install: Timed out while waiting for response to ClientRequest. Waited 5.0 seconds.